## Set up basic libraries

In [ ]:
import os
from IPython.core.display import display, HTML
print(os.getcwd())
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
import matplotlib
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
import numpy as np
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import os
import rasterio
from rasterio.transform import from_origin
from rasterio import features
%matplotlib inline
import sys, os
sys.path.insert(0,"../..")
print(os.getcwd())
rootdir=r'C:\Users\00073294\Dropbox\1_Jupyter_notebooks\map2model\\'
os.chdir(rootdir)
print(os.getcwd())

In [ ]:
mname='hams2' #root name of model input data

print(os.getcwd())
newdir=rootdir+mname
print(newdir)
os.chdir(newdir)

if not os.path.exists(os.path.isdir('tmp')): 
    os.mkdir('tmp')
    os.mkdir('gempy')
    os.mkdir('dtm')

text=False    #display debug text
graphics=True #display debug images


model_base=-8200 # top could be found using highest point in topo data

minlong=117
maxlong=118
minlat=-23
maxlat=-22

minx=500057    #left hams
maxx=603028    #right
miny=7455348   #bottom
maxy=7567953   #top
    

step_out=0.045 #add (in degrees) so edge pixel from dtm reprojection are not found

src_crs = 'epsg:2346'  #input data coord ref system (assumed to be geodetic lat/long WGS84)
dst_crs = 'epsg:28350' #model ref system (assumed to be something with metre coordinates)

# 'epsg:28350'  =  GDA_1994_MGA_Zone_50 
# 'epsg:4326'   =   wgs84 GEODETIC LAT/LONG 

orientation_decimate=0   # decimation of orientaiton data (0 = no decimation)
contact_decimate=30   # decimation of contact data (0 = no decimation)

gcode='GROUP_'
dcode='DIP'
ddcode='DIP_DIR'
ccode='CODE'


## Load geology map

In [ ]:
# Extract point data from WAROX & geology map for gempy
##First we readin the WAROX and map from shapefiles, or wherever...

import geopandas as gpd
import os
from shapely.geometry import Polygon
from shapely.geometry import Point


%matplotlib inline
os.environ["PROJ_LIB"] = r"C:\Users\00073294\AppData\Local\Continuum\anaconda3\Lib\site-packages\pyproj\proj_dir\share\proj"


geology = gpd.read_file('.\\data\\geol_clip.shp')
print(geology.crs)


## Basic explode code

In [ ]:
#https://gist.github.com/mhweber/cf36bb4e09df9deee5eb54dc6be74d26
    
import geopandas as gpd
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

def explode(indf):
#    indf = gpd.GeoDataFrame.from_file(indata)
    outdf = gpd.GeoDataFrame(columns=indf.columns)
    for idx, row in indf.iterrows():
        if type(row.geometry) == Polygon:
            outdf = outdf.append(row,ignore_index=True)
        if type(row.geometry) == MultiPolygon:
            multdf = gpd.GeoDataFrame(columns=indf.columns)
            recs = len(row.geometry)
            multdf = multdf.append([row]*recs,ignore_index=True)
            for geom in range(recs):
                multdf.loc[geom,'geometry'] = row.geometry[geom]
            outdf = outdf.append(multdf,ignore_index=True)
    return outdf

## Load topological sorted surfaces

In [ ]:
ag=open('.\\tmp\\'+mname+'_all_sorts.txt',"r")
contents =ag.readlines()
ag.close

ulist=[]
for i in range(1,len(contents)):
    cont_list=contents[i].split(",")
    ulist.append([i, cont_list[4].replace("\n","")])
print(ulist)


## Load all basal contact points

In [ ]:
import math
def angle(pt1, pt2):
    x1, y1 = pt1
    x2, y2 = pt2
    inner_product = x1*x2 + y1*y2
    len1 = math.hypot(x1, y1)
    len2 = math.hypot(x2, y2)
    dot=math.degrees(math.acos(inner_product/(len1*len2)))
    return 90-math.fabs(90-dot)

def pairs(lst):
    for i in range(1, len(lst)):
        yield lst[i-1], lst[i]

        #https://www.geeksforgeeks.org/program-check-three-points-collinear/

# Python program to check 
# if three points are collinear 
# or not using area of triangle. 
  
# function to check if  
# point collinear or not 
def collinear(x1, y1, x2, y2, x3, y3): 
      
    """ Calculation the area of   
        triangle. We have skipped  
        multiplication with 0.5 to 
        avoid floating point computations """
    a = x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2) 
    b=x1 * (y2 - y3)
    c=x2 * (y3 - y1)
    d=x3 * (y1 - y2) 
  
    #if (a < 0.000001): 
    #    print ("Yes",a,b,c,d)
    #else: 
    #    print ("No",a,b,c,d)
    return(a< 0.000001)
  
# This code is contributed 
# by Sachin Bisht 

import rasterio

dataset = rasterio.open('.\\dtm\\'+mname+'_dtm_rp.tif')

def height_from_srtm(x,y):
    locations=[(x,y)]
    for val in dataset.sample(locations):
        return(str(val).replace("[","").replace("]",""))


## Create basal contact points with orientation from orientations and basal points

In [ ]:
import pandas as pd
from geopandas import GeoDataFrame
from shapely.geometry import MultiPoint,Point,LineString
from shapely.ops import nearest_points
import math

contacts = gpd.read_file('./tmp/basal_contacts2.shp') #load orientation data as geopandas dataframe 

warox = gpd.read_file('.\\data\\warox_clip.shp') #load orientation data as geopandas dataframe 

contacts.plot(figsize=(10,10),edgecolor='#000000',linewidth=0.2)

In [ ]:
f=open('.\\gempy\\'+mname+'_projected_dip_contacts2.txt',"w")
f.write('id,x,y,z,DIP,DIP_DIR,polarity,CODE,GROUP_\n')
#print("len=",len(contacts))
i=0
for acontact in contacts.iterrows():   #loop through distinct linestrings
    #display(acontact[1].geometry)
    thegroup=acontact[1]['GROUP_']
    #print("thegroup=",thegroup)
    is_wrx=warox['GROUP_'] == thegroup # subset orientations to just those with this group
    all_wrx = warox[is_wrx]

    for awrx in all_wrx.iterrows(): # loop through valid orientations
        #print(awrx[1].geometry,awrx[1]['GROUP_'])
        orig = Point(awrx[1]['geometry'])
        np = acontact[1].geometry.interpolate(acontact[1].geometry.project(orig))
        if(np.distance(orig)<5000):
            #print(acontact[1]['CODE'],awrx[1]['DIP'],orig,np)                  
            for line in acontact[1].geometry: # loop through line segments
                for pair in pairs(list(line.coords)): # loop through line segments
                    segpair=LineString((pair[0],pair[1]))
                    if segpair.distance(np)< 0.0001: # line segment closest to close point
                        ddx=math.sin(math.radians(awrx[1]['DIP_DIR']))
                        ddy=math.cos(math.radians(awrx[1]['DIP_DIR']))
                        dlsx=pair[0][0]-pair[1][0]
                        dlsy=pair[0][1]-pair[1][1]
                        lsx=dlsx/math.sqrt((dlsx*dlsx)+(dlsy*dlsy))
                        lsy=dlsy/math.sqrt((dlsx*dlsx)+(dlsy*dlsy))
                        angle=math.degrees(math.acos((ddx*lsx)+(ddy*lsy)))
                        #print(i,angle,ls_ddir,lsx,lsy,awrx[1]['DIP_DIR'])
                        if(math.fabs(angle-90)<30.0): # dip_dir normal and contact are close enough to parallel
                            height= height_from_srtm(np.x,np.y)
                            ls_ddir=math.degrees(math.atan2(lsy,-lsx)) #normal to line segment
                            #print(i,ddx,ddy,lsx,lsy,ls_ddir,awrx[1]['DIP_DIR'])
                           
                            if (ddx*lsy)+(-ddy*lsx)<0: #dot product tests right quadrant
                                ls_ddir=(ls_ddir-180)%360
                            ostr=str(i)+","+str(np.x)+","+str(np.y)+","+height+","+str(awrx[1]['DIP'])+","+str(ls_ddir)+",1,"+acontact[1]['CODE']+","+acontact[1]['GROUP_']+"\n" 
                            f.write(ostr)
                            i=i+1


f.close()

## Estimate layer thicknesses

In [ ]:
warox = gpd.read_file('./data/warox_clip.shp') #load orientation data as geopandas dataframe 
contacts = gpd.read_file('./tmp/basal_contacts2.shp') #load orientation data as geopandas dataframe 

f=open('./gempy/'+mname+'_estimated_layer_thicknesses.txt',"w")
f.write('id,x,y,CODE,thickness\n')

all_sorts=pd.read_csv('./tmp/'+mname+'_all_sorts.txt')
#display(all_sorts)
i=0
print("id,x,y,code,group")
for apair in all_sorts.iterrows():
    for bpair in all_sorts.iterrows():
        if(int(apair[1]['index'])==(int(bpair[1]['index'])+1) and apair[1]['group']==bpair[1]['group'] ):
            #print(apair[1]['code'],apair[1]['group'])
            #print(warox.shape)
            is_wrx=warox['GROUP_'] == apair[1]['group'] # subset orientations to just those with this group
            all_wrx = warox[is_wrx]      
            #print(all_wrx.shape)
            is_contacta=contacts['CODE'] == apair[1]['code'] # subset contacts to just those with a code
            acontacts = contacts[is_contacta]
            
            is_contactb=contacts['CODE'] == bpair[1]['code'] # subset contacts to just those with b code
            bcontacts = contacts[is_contactb]

            for awrx in all_wrx.iterrows(): # loop through valid orientations
                
                dx1=math.sin(math.radians(awrx[1]['DIP_DIR']))*1e5
                dy1=math.cos(math.radians(awrx[1]['DIP_DIR']))*1e5
                dx2=-dx1
                dy2=-dy1
                p1=Point((dx1+awrx[1]['geometry'].x,dy1+awrx[1]['geometry'].y))
                p2=Point((dx2+awrx[1]['geometry'].x,dy2+awrx[1]['geometry'].y))
                ddline=LineString((p1,p2))
                orig = Point(awrx[1]['geometry'])

                for acontact in acontacts.iterrows():   #loop through distinct linestrings for upper contact
                    #print(acontact[1]['CODE'],apair[1]['code'])
                    #break
                    if(ddline.intersects(acontact[1].geometry)): 
                        isects=ddline.intersection(acontact[1].geometry)
                        #print(type(isects),isects)
                        if(isects.geom_type=="MultiPoint"):
                            for pt in isects: 
                                if(pt.distance(orig)<1000):
                                    print(i,",", pt.x, ",",pt.y,",",apair[1]['code'],",",apair[1]['group'])
                                    i=i+1
                        else:
                            if(isects.distance(orig)<1000):
                                print(i,",", isects.x,",", isects.y,",",apair[1]['code'],",",apair[1]['group'])
                                i=i+1
                
                for bcontact in bcontacts.iterrows():   #loop through distinct linestrings for lower contact
                    #break
                    if(ddline.intersects(bcontact[1].geometry)): 
                        isects=ddline.intersection(bcontact[1].geometry)
                        if(isects.geom_type=="MultiPoint"):
                            for pt in isects: 
                                if(pt.distance(orig)<15000):
                                    print(i,",", pt.x, ",",pt.y,",",bpair[1]['code'],",",bpair[1]['group'])
                                    i=i+1
                        else:
                            if(isects.distance(orig)<15000):
                                print(i,",", isects.x,",", isects.y,",",bpair[1]['code'],",",bpair[1]['group'])
                                i=i+1
            

In [ ]:
                    
                    np = acontact[1].geometry.interpolate(acontact[1].geometry.project(orig))
                    if(np.distance(orig)<1000):
                        #print(acontact[1]['CODE'],awrx[1]['DIP'],orig,np)                  
                        for line in acontact[1].geometry: # loop through line segments
                            for pair in pairs(list(line.coords)): # loop through line segments
                                segpair=LineString((pair[0],pair[1]))
                                if segpair.distance(np)< 0.0001: # line segment closest to close point
                                    ddx=math.sin(math.radians(awrx[1]['DIP_DIR']))
                                    ddy=math.cos(math.radians(awrx[1]['DIP_DIR']))
                                    dlsx=pair[0][0]-pair[1][0]
                                    dlsy=pair[0][1]-pair[1][1]
                                    lsx=dlsx/math.sqrt((dlsx*dlsx)+(dlsy*dlsy))
                                    lsy=dlsy/math.sqrt((dlsx*dlsx)+(dlsy*dlsy))
                                    angle=math.degrees(math.acos((ddx*lsx)+(ddy*lsy)))
                                    #print(i,angle,ls_ddir,lsx,lsy,awrx[1]['DIP_DIR'])
                                    if(math.fabs(angle-90)<30.0): # # dip_dir normal and contact are close enough to parallel
                                        print(orig,np)
                                        bigddx=ddx*1.e5
                                        bigddy=ddy*1.e5
        

In [ ]:
for awrx in warox.iterrows(): # loop through orientations
    no_xing=0
    xinglist={}
    orig = Point(awrx[1]['geometry'])
    thegroup=awrx[1]['GROUP_']
    
    is_contact=contacts['GROUP_'] == thegroup # subset orientations to just those with this group
    all_contacts = contacts[is_contact]
    
    for acontact in all_contacts.iterrows():   #loop through distinct linestrings of same group
        #print(i,acontact[1]['GROUP_'],awrx[1]['GROUP_'])
        dx1=math.sin(math.radians(awrx[1]['DIP_DIR']))*1e5
        dy1=math.cos(math.radians(awrx[1]['DIP_DIR']))*1e5
        dx2=-dx1
        dy2=-dy1
        p1=Point((dx1+awrx[1]['geometry'].x,dy1+awrx[1]['geometry'].y))
        p2=Point((dx2+awrx[1]['geometry'].x,dy2+awrx[1]['geometry'].y))
        ddline=LineString((p1,p2))
        #print(i,ddline)
        
        #for acontactline in acontact[1].geometry:
        if ddline.intersects(acontact[1].geometry): 
            #print(i,ddline,acontactline)
            xing=acontact[1].geometry.intersection(ddline)
            
            print(acontact[1]['CODE'])
            #f.write(str(xing)+"\n")
            no_xing=no_xing+1
    
    if(no_xing>1):
        print(i,no_xing,"------------------\n")
        for xi in xinglist:
            print(xi)
    i=i+1


f.close()

In [ ]:
    is_anid=contacts['id'] == anid
    acontact = contacts[is_anid] # subset contacts to just those with this id    
    #print("arow",arow,len(contacts))
    thegroup=acontact.iloc[0]['GROUP_'] #find first rows group, will be same for all points
    #print("arow=",arow,"anid=",anid,"thegroup=",thegroup,"trial=",acontact.iloc[0]['GROUP_'])   
    apl=LineString(list(acontact.geometry)) #create lineString from points
    display(apl)

    is_wrx=warox['GROUP_'] == thegroup # subset orientations to just those with this group
    all_wrx = warox[is_wrx]

    for awrx in all_wrx.iterrows(): # loop through valid orientations
        orig = Point(awrx[1]['geometry'])
        #display(orig)
        dx1=math.sin(math.radians(awrx[1]['DIP_DIR']))*1e5
        dy1=math.cos(math.radians(awrx[1]['DIP_DIR']))*1e5
        dx2=-dx1
        dy2=-dy1
        p1=Point((dx1+awrx[1]['geometry'].x,dy1+awrx[1]['geometry'].y))
        p2=Point((dx2+awrx[1]['geometry'].x,dy2+awrx[1]['geometry'].y))
        print("i,dd,dx1,dy1,p1,dx2,dy2,p2\n",i,awrx[1]['DIP_DIR'],dx1,dy1,p1,dx2,dy2,p2)
        ddline=LineString((p1,p2))
        #display(ddline)
        #print(awrx[1]['DIP_DIR'],ddline)
        if(ddline.intersects(apl)): 
            isects=ddline.intersection(apl)
            #print(isects)
            nearest_geom = nearest_points(orig, isects)
            #print("NG orig closest",nearest_geom,len(nearest_geom))
            if(nearest_geom[0].distance(nearest_geom[1])<250000): #only for close matches (in metres)
                for pair in pairs(list(apl.coords)): # loop through line segments
                    ls1=((pair[0][0]-pair[1][0],pair[0][1]-pair[1][1])) #line segment vector
                    lsa=Point((pair[0][0],pair[0][1]))
                    lsb=Point((pair[1][0],pair[1][1]))
                    lsc=LineString((lsa,lsb))
                    #display(lsc)
                    #print(ls1)
                    ddn1=((dx1,dy1)) #  dip direction vector
                    #print(angle(ddn1,ls1))
                    if collinear(pair[0][0],pair[0][1],pair[1][0],pair[1][1],nearest_geom[1].x,nearest_geom[1].y): # line segment closest to close point
                        if(angle(ddn1,ls1)<20.0): #(angle between strike and contact less than xx degrees)
                            lsvector=ls1[0]/math.sqrt((ls1[0]*ls1[0])+(ls1[1]*ls1[1]))
                            ostr=str(i)+","+str(nearest_geom[1].x)+","+str(nearest_geom[1].y)+",99,1,"+str(awrx[1]['DIP'])+","+str(180+math.degrees(math.asin(lsvector)))+",1,"+acontact.iloc[0]['code']+","+str(angle(ddn1,ls1))+"\n"
                                #","+str(awrx[1]['DIP_DIR'])+",1,"+acontact.iloc[0]['code']+","+str(angle(ddn1,ls1))+"\n"
                            
                            #back calc of dip dir from segment broken
                            #angle between ddir and segment broken (maybe segment broken!)
                            
                            #print("----",i,pair[0],pair[1],ls1,awrx[1]['DIP_DIR'],ddn1,angle(ddn1,ls1))
                            f.write(ostr)
                            i=i+1
                            break
        break
f.close()

warox = gpd.read_file('.\\data\\warox_clip.shp') #load orientation data as geopandas dataframe 

f=open('.\\gempy\\'+mname+'_projected_dip_contacts2.txt',"w")
f.write('id,x,y,z,DIP,DIP_DIR,polarity,code,angle\n')
#print("len=",len(contacts))
uids=(contacts['id'].unique())
#print(uids)
i=0
for anid in uids:   #loop through distinct poly ids
    #print("plid=",anid)  
    
    is_anid=contacts['id'] == anid
    acontact = contacts[is_anid] # subset contacts to just those with this id    
    #print("arow",arow,len(contacts))
    thegroup=acontact.iloc[0]['GROUP_'] #find first rows group, will be same for all points
    #print("arow=",arow,"anid=",anid,"thegroup=",thegroup,"trial=",acontact.iloc[0]['GROUP_'])   
    apl=LineString(list(acontact.geometry)) #create lineString from points
    display(apl)

    is_wrx=warox['GROUP_'] == thegroup # subset orientations to just those with this group
    all_wrx = warox[is_wrx]

    for awrx in all_wrx.iterrows(): # loop through valid orientations
        orig = Point(awrx[1]['geometry'])
        #display(orig)
        dx1=math.sin(math.radians(awrx[1]['DIP_DIR']))*1e5
        dy1=math.cos(math.radians(awrx[1]['DIP_DIR']))*1e5
        dx2=-dx1
        dy2=-dy1
        p1=Point((dx1+awrx[1]['geometry'].x,dy1+awrx[1]['geometry'].y))
        p2=Point((dx2+awrx[1]['geometry'].x,dy2+awrx[1]['geometry'].y))
        print("i,dd,dx1,dy1,p1,dx2,dy2,p2\n",i,awrx[1]['DIP_DIR'],dx1,dy1,p1,dx2,dy2,p2)
        ddline=LineString((p1,p2))
        #display(ddline)
        #print(awrx[1]['DIP_DIR'],ddline)
        if(ddline.intersects(apl)): 
            isects=ddline.intersection(apl)
            #print(isects)
            nearest_geom = nearest_points(orig, isects)
            #print("NG orig closest",nearest_geom,len(nearest_geom))
            if(nearest_geom[0].distance(nearest_geom[1])<250000): #only for close matches (in metres)
                for pair in pairs(list(apl.coords)): # loop through line segments
                    ls1=((pair[0][0]-pair[1][0],pair[0][1]-pair[1][1])) #line segment vector
                    lsa=Point((pair[0][0],pair[0][1]))
                    lsb=Point((pair[1][0],pair[1][1]))
                    lsc=LineString((lsa,lsb))
                    #display(lsc)
                    #print(ls1)
                    ddn1=((dx1,dy1)) #  dip direction vector
                    #print(angle(ddn1,ls1))
                    if collinear(pair[0][0],pair[0][1],pair[1][0],pair[1][1],nearest_geom[1].x,nearest_geom[1].y): # line segment closest to close point
                        if(angle(ddn1,ls1)<20.0): #(angle between strike and contact less than xx degrees)
                            lsvector=ls1[0]/math.sqrt((ls1[0]*ls1[0])+(ls1[1]*ls1[1]))
                            ostr=str(i)+","+str(nearest_geom[1].x)+","+str(nearest_geom[1].y)+",99,"+str(awrx[1]['DIP'])+","+str(180+math.degrees(math.asin(lsvector)))+",1,"+acontact.iloc[0]['code']+","+str(angle(ddn1,ls1))+"\n"
                                #","+str(awrx[1]['DIP_DIR'])+",1,"+acontact.iloc[0]['code']+","+str(angle(ddn1,ls1))+"\n"
                            
                            #back calc of dip dir from segment broken
                            #angle between ddir and segment broken (maybe segment broken!)
                            
                            #print("----",i,pair[0],pair[1],ls1,awrx[1]['DIP_DIR'],ddn1,angle(ddn1,ls1))
                            f.write(ostr)
                            i=i+1
                            break
        break
f.close()

## Project orientations to all closest contacts of same group

In [ ]:
for all orientations
    calculate very long dip direction line
        find closest point in group of one polyid

## Create summary polygon info

In [ ]:
#df = pd.DataFrame(columns=['ID', 'COORDS', 'OBJECTID_1','CODE','GROUP_','PLENGTH','PAREA','RELAGE','geometry'])

poly_info=[]
i=0
for  apoly in geology.iterrows():

    apoly_geom=apoly[1].geometry
    out=[item for item in ulist if apoly[1]['CODE'] in item]
    if(len(out)>0):
        #print("relage",out[0][0])
        print("code",out[0][0],apoly[1]['CODE'])
        if(str(apoly[1]['GROUP_'])   =="None"):
            pgroup=apoly[1]['CODE']
        else:
            pgroup=apoly[1]['GROUP_']
        
        poly_info.append([i,apoly_geom.bounds,(apoly[1]['geometry'].centroid.x,apoly[1]['geometry'].centroid.y),
            apoly[1]['OBJECTID_1'],apoly[1]['CODE'],pgroup,apoly[1]['geometry'].length,
            apoly[1]['geometry'].area,out[0][0], apoly[1].geometry])
    i=i+1
    
display(poly_info)

## Load warox point data

In [ ]:
warox = gpd.read_file('.\\data\\warox_clip.shp')
for apoint in warox.iterrows():
        print("pts",apoint)
        break

## Find closest polygon one unit younger of same group and orientation point that is of same group and is closest to chosen polygon

In [ ]:
from shapely.geometry import Polygon, mapping
#is_2002 =  gapminder['year']==2002

#filter polygons for a group_, filter warox for a group_ 

f=open('.\\tmp\\'+mname+'_groups2.txt',"r")
contents =f.readlines()
f.close

ngroups=contents[0].split(" ")
ngroups=int(ngroups[1])

groups=[]
for i in range (1,int(ngroups)+1):
    #print(contents[i].replace("\n",""))
    groups.append((contents[i].replace("\n","")))

print(ngroups,groups)


In [ ]:
import pandas as pd
def write_group_poly_info(geology,ccode,gcode,idx):
    
    rows_list = []
      
    i=0
    for  apoly in geology.iterrows():

        apoly_geom=apoly[1].geometry
        out=[item for item in ulist if apoly[1][ccode] in item]
        if(len(out)>0):
            #print("relage",out[0][0])
            #print("relage:",out[0][0],apoly[1][ccode],apoly[1][id])
            if(str(apoly[1][gcode])   =="None"):
                pgroup=apoly[1][ccode]
            else:
                pgroup=apoly[1][gcode]

        rows_list.append({'ID': i, 'COORDSX': apoly[1]['geometry'].centroid.x, 'COORDSY':apoly[1]['geometry'].centroid.y,
            'OBJECTID_1':apoly[1][idx],'CODE':apoly[1][ccode],'GROUP_':pgroup,'PLENGTH':apoly[1]['geometry'].length,
            'PAREA':apoly[1]['geometry'].area,'RELAGE':out[0][0]})

        i=i+1

    poly_info=pd.DataFrame(rows_list) 
    #display(poly_info)
    poly_info.to_csv("tmp\\poly_info.txt",index=False)
    
    return

In [ ]:
def find_orientation_for_surface(geol,warox):
                
    c=open('./tmp/close_pts.txt','w')
    c.write('x,y,dip,dipdir\n')
    ostr=[]
    i=0
    for apoly in geol.iterrows(): # for each poly
        min_dist=1e10
        for bpoly in geol.iterrows(): #find closest polygon 1 younger and same group
            print(apoly[1]['OBJECTID_1'],bpoly[1]['OBJECTID_1'])
            if(not apoly[0] == bpoly[0]):
                print(apoly[0],bpoly[0])
                print(apoly[2][1])
                print(bpoly[2][0:1])
                print(bpoly[2][1:2])
                dist=((apoly[2][0]-bpoly[2][0])*(apoly[2][0]-bpoly[2][0]))+((apoly[2][1]-bpoly[2][1])*(apoly[2][1]-bpoly[2][1]))
                if(dist<min_dist and apoly[5]==bpoly[5] and not apoly[4]==bpoly[4] and apoly[8]==(bpoly[8]+1) ):
                    min_dist=dist
                    min_poly=bpoly                

    for apoly in geol: # for each poly
        x= mapping(apoly[9])
        min_pt_dist=1e10
        for ppts in x['coordinates'][0]: #for target polygon find closest structure data of same group
            for apoint in warox.iterrows():            
                if(len(ppts)==2 and apoint[1]['GROUP_']==apoly[5]): # only single point and right group
                    p1=float(apoint[1].geometry.x)-ppts[0]
                    p2=float(apoint[1].geometry.y)-ppts[1]
                    dist=(p1*p1)+(p2*p2)
                    if(dist < min_pt_dist): #new closest point
                        min_dist=dist
                        min_pt=apoint                
        #print(dist,min_pt[1]['DIP'],min_pt[1]['DIP_DIR'])

        ostr=(str(apoint[1].geometry.x)+','+str(apoint[1].geometry.y)+','+str(min_pt[1]['DIP'])+','+str(min_pt[1]['DIP_DIR'])+'\n')
        c.write(ostr)
        print(i)
        #break
        i+=1
    c.close()  
    return


In [ ]:
for i in range(0,ngroups):
    g=open('.\\tmp\\'+groups[i].replace(" ","_")+'.txt',"r")
    contents =g.readlines()
    g.close
    hdr=contents[0].split(" ")
    nstrat=int(hdr[1])
    
    is_group = geology['GROUP_']==groups[i]
    group_geol = geology[is_group]
    write_group_poly_info(group_geol,'CODE','GROUP_','OBJECTID_1')
    print(groups[i],len(geology),len(group_geol))
    is_group = warox['GROUP_']==groups[i]
    group_warox = warox[is_group]
    #print("warox",len(warox),len(group_warox))
    if(nstrat>1):
        for j in range (2,nstrat+1):
            is_code = geology['CODE']==contents[j].replace("\n","")
            code_geol = geology[is_code]
            print(contents[j].replace("\n",""),len(group_warox),len(code_geol))
            find_orientation_for_surface(group_geol,group_warox)
    else:
        print("no need")

In [ ]:
display(group_geol)

In [ ]:
from shapely.geometry import Polygon, mapping

c=open('./tmp/close_pts.txt','w')
c.write('x,y,dip,dipdir\n')
ostr=[]
i=0
for apoly in poly_info: # for each poly
    min_dist=1e10
    for bpoly in poly_info: #find closest polygon 1 younger and same group
        if(not apoly[0] == bpoly[0]):
            dist=((apoly[2][0]-bpoly[2][0])*(apoly[2][0]-bpoly[2][0]))+((apoly[2][1]-bpoly[2][1])*(apoly[2][1]-bpoly[2][1]))
            if(dist<min_dist and apoly[5]==bpoly[5] and not apoly[4]==bpoly[4] and apoly[8]==(bpoly[8]+1) ):
                min_dist=dist
                min_poly=bpoly                

for apoly in poly_info: # for each poly
    x= mapping(apoly[9])
    min_pt_dist=1e10
    for ppts in x['coordinates'][0]: #for target polygon find closest structure data of same group
        for apoint in warox.iterrows():            
            if(len(ppts)==2 and apoint[1]['GROUP_']==apoly[5]): # only single point and right group
                p1=float(apoint[1].geometry.x)-ppts[0]
                p2=float(apoint[1].geometry.y)-ppts[1]
                dist=(p1*p1)+(p2*p2)
                if(dist < min_pt_dist): #new closest point
                    min_dist=dist
                    min_pt=apoint                
    #print(dist,min_pt[1]['DIP'],min_pt[1]['DIP_DIR'])
    
    if(min_pt_dist<1e9):
        ostr=(str(apoint[1].geometry.x)+','+str(apoint[1].geometry.y)+','+str(min_pt[1]['DIP'])+','+str(min_pt[1]['DIP_DIR'])+'\n')
        c.write(ostr)
    print(i)
    #break
    i+=1
c.close()
           